In [ ]:
# imports

import matplotlib as mpl
import matplotlib.colors as mcolors
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import pycmap
import seaborn as sns
from tlviz.factor_tools import factor_match_score
import xarray as xr
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)


In [ ]:
# functions to build continuous colormap
# source: https://towardsdatascience.com/beautiful-custom-colormaps-with-matplotlib-5bab3d1f0e72

def hex_to_rgb(value):
    '''
    Converts hex to rgb colours
    value: string of 6 characters representing a hex colour.
    Returns: list length 3 of RGB values'''
    value = value.strip("#") # removes hash symbol if present
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))


def rgb_to_dec(value):
    '''
    Converts rgb to decimal colours (i.e. divides each value by 256)
    value: list (length 3) of RGB values
    Returns: list (length 3) of decimal values'''
    return [v/256 for v in value]

def get_continuous_cmap(hex_list, float_list=None):
    ''' creates and returns a color map that can be used in heat map figures.
        If float_list is not provided, colour map graduates linearly between each color in hex_list.
        If float_list is provided, each color in hex_list is mapped to the respective location in float_list. 
        
        Parameters
        ----------
        hex_list: list of hex code strings
        float_list: list of floats between 0 and 1, same length as hex_list. Must start with 0 and end with 1.
        
        Returns
        ----------
        colour map'''
    rgb_list = [rgb_to_dec(hex_to_rgb(i)) for i in hex_list]
    if float_list:
        pass
    else:
        float_list = list(np.linspace(0,1,len(rgb_list)))
        
    cdict = dict()
    for num, col in enumerate(['red', 'green', 'blue']):
        col_list = [[float_list[i], rgb_list[i][num], rgb_list[i][num]] for i in range(len(float_list))]
        cdict[col] = col_list
    cmp = mcolors.LinearSegmentedColormap('my_cmp', segmentdata=cdict, N=256)
    return cmp
    

In [ ]:
# set aesthetic parameters

line_color = '#0F0A0A'    # dark
# line_color = '#E5E4E2'    # light

neutral_color = '#E0E0E0'

# thursday
accent_colors = ['#9B5DE5', '#FFAC69', '#00C9AE', '#FD3F92', '#0F0A0A', 
                 '#959AB1', '#FFDB66', '#63B9FF','#FFB1CA', '#4F1DD7']

grays = ['#0F0A0A', '#52525E', '#747688', '#959AB1', '#959AB1', '#CECFD5', '#E0E0E0']

style = {'axes.edgecolor': line_color,
         'axes.labelcolor': line_color,
         'text.color': line_color,
         'xtick.color': line_color,
         'ytick.color': line_color,
         'font.family': 'Helvetica',
         'font.Helvetica': ['Helvetica']}

palette = sns.color_palette(accent_colors)

sns.set_context('talk', rc={'lines.linewidth': 2})
sns.set_palette(palette)
# sns.set_palette('tab20')
sns.set_style('ticks', style)

plt.rcParams['legend.frameon'] = False
plt.rcParams['axes.spines.left'] = True
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.bottom'] = True
plt.rcParams['axes.facecolor'] = 'none'
plt.rcParams['figure.facecolor'] = 'none'
plt.rcParams['patch.linewidth'] = 0
plt.rcParams['patch.edgecolor'] = 'none'
plt.rcParams['savefig.dpi'] = 300

# bi_hexes = [neutral_color, '#7222D3']
bi_hexes = [neutral_color, line_color]
bi_cmap = get_continuous_cmap(bi_hexes)


In [ ]:
# helper functions

def unique_bins(array, n_bins, bounds=None, reverse=False):
    '''This is a function that maps an array of continuous values with potential duplicates to 
    a set of bins in such a way that each of the original continuous values is mapped to a unique bin, and the
    distribution of binned values approximates the distribution of continuous values as closely as possible.
    
    My quick and dirty approach to this problem sorts the input array into the appropriate number of bins. Then,
    starting in the middle and alternating outward left and right, the algorithm checks to see if each bin has 
    more than one resident. If so, it will again alternate outward left and right from that position and allocate
    the extra residents to the nearest free bins.
    
    Parameters
    ----------
    array : np.array
        Sorted array of values to be placed into bins.
    n_bins : int
        Number of bins to use.
    bounds : (float, float)
        Upper and lower bounds to use for bin-mapping.
    reverse : bool
        Optional flag to reverse the index ordering of the output.
    
        
    Returns
    -------
    index : np.array
        Indices indicating mapping of input values to bins, where index[i] indicates the index of the bin 
        mapping of input value array[i] (assuming input array is sorted).
    '''
    # check number of bins
    if not n_bins >= len(array):
        raise ValueError('The number of bins must be >= len(array).')
    # add bounds if supplied
    if bounds is not None:
        array = np.concatenate((array, bounds))
    # make histogram
    counts, _ = np.histogram(array, bins=n_bins)
    # remove bounds if added
    if bounds is not None:
        counts[0] -= 1
        counts[-1] -= 1
    # start from the middle
    i = int(n_bins/2)
    for j in range(n_bins):
        # alternate left and right
        if j % 2:
            i -= j
        else:
            i += j
        # resolve collisions
        if counts[i] > 1:
            # set new counter for exploring nearby positions
            k = i
            for l in range(2*n_bins):
                # alternate left and right
                if l % 2:
                    k -= l
                else:
                    k += l
                # check if we're still in range
                if 0 <= k < n_bins:
                    # check for an empty spot
                    if counts[k] == 0:
                        counts[k] = 1
                        counts[i] -= 1
                        # check if all collisions have been resolved yet
                        if counts[i] == 1:
                            break
    # select indices of non-zero values
    index = np.where(counts == 1)[0]
    # reverse if requested
    if reverse:
        index = (n_bins - 1) - index
    return index


# import data
### cluster data

In [ ]:
# read in aligned model datasets

datapath = Path('../../data/')
outdir = Path('../../figures/7-interpretation/')

pro_ds = xr.open_dataset(datapath / '5-models/pro-aligned-models.nc')
syn_ds = xr.open_dataset(datapath / '5-models/syn-aligned-models.nc')

fe_clusters = {'pro': 7, 'syn': 8}
n_clusters = {'pro': 15, 'syn': 12}

pro_ds


In [ ]:
# read in metadata

sample_df = pd.read_csv(datapath / 'metadata/sample_metadata_merged.csv')
sample_df['Datetime'] = pd.to_datetime(sample_df['Datetime'], format='%m/%d/%y %H:%M')

sample_df


In [ ]:
# pull out clusters hypothesized to relate to iron limitation

pro_fe_df = pro_ds.sel(Component=fe_clusters['pro']).SampleWeight.to_series().reset_index()
pro_fe_df['Genus'] = 'Prochlorococcus'
syn_fe_df = syn_ds.sel(Component=fe_clusters['syn']).SampleWeight.to_series().reset_index()
syn_fe_df['Genus'] = 'Synechococcus'

fe_cluster_df = pd.concat([pro_fe_df, syn_fe_df]).reset_index(drop=True)
fe_cluster_df = pd.merge(fe_cluster_df, sample_df, on='SampleName', how='left')

fe_cluster_df


In [ ]:
# pull out clusters hypothesized to relate to nitrogen limitation

pro_n_df = pro_ds.sel(Component=n_clusters['pro']).SampleWeight.to_series().reset_index()
pro_n_df['Genus'] = 'Prochlorococcus'
syn_n_df = syn_ds.sel(Component=n_clusters['syn']).SampleWeight.to_series().reset_index()
syn_n_df['Genus'] = 'Synechococcus'

n_cluster_df = pd.concat([pro_n_df, syn_n_df]).reset_index(drop=True)
n_cluster_df = pd.merge(n_cluster_df, sample_df, on='SampleName', how='left')

n_cluster_df

In [ ]:
# read in gene annotations

gene_names_df = pd.read_excel(
    datapath / '7-interpretation/fe-n-cluster-annotations.xlsx', 
    sheet_name='Fe-N-CyCOGs', 
    usecols=['Ortholog', 'Gene Name', 'Protein Name', 'Expression Response']
).rename(columns={'Gene Name': 'GeneLabel', 'Protein Name': 'ProteinLabel', 'Expression Response': 'ExpressionResponse'})

gene_names_df


# import data
### environmental data

In [ ]:
# define salinity and chlorophyll fronts

# source: Juranek et al., 2020 https://doi.org/10.1029/2020GB006702
sal_fronts = {2016: 32.15, 2017: 32.5, 2019: 32.34}
chl_fronts = {2016: 33, 2017: 36.2, 2019: 35}


In [ ]:
# import data from cmap

api = pycmap.API(token='55bd80f9-8d5d-4c8f-8917-f5b0e924ed45')
api.search_catalog('iron John gradients')


In [ ]:
# import all iron data from G1, G2, G3

fe_df = pd.concat([
    api.get_dataset('tblKOK1606_Gradients1_Diss_Trace_Metal_Profile'), 
    api.get_dataset('tblMGL1704_Gradients2_Diss_Trace_Metal_Profile'), 
    api.get_dataset('tblMGL1704_Gradients2_Diss_Trace_Metal_Surface'), 
    api.get_dataset('tblKM1906_Gradients3_Diss_Trace_Metal_Profile')
]).reset_index(drop=True)
fe_df['year'] = pd.to_datetime(fe_df['time']).dt.year

fe_df

In [ ]:
# import nitrogen data from G1, G2, G3

# dissolved nitrate+nitrite (Karl lab)
n_df =  pd.concat([
    api.get_dataset('tblKOK1606_Gradients1_Nutrients'), 
    api.get_dataset('tblMGL1704_Gradients2_Nutrients'), 
    api.get_dataset('tblKM1906_Gradients3_Organic_Inorganic_Nutrients').rename(columns={'NO3_plus_NO2': 'NO3_NO2'})
]).reset_index(drop=True)
n_df['year'] = pd.to_datetime(n_df['time']).dt.year

# drop any rows that don't have nitrate+nitrite values
n_df = n_df[n_df['NO3_NO2'].notna()]

n_df


In [ ]:
# merge Fe data on dissolved N data

# only select surface and depth samples for joining
source_df = n_df[n_df.depth.le(15)].reset_index(drop=True)
source_cols = source_df.columns.to_list()
tolerances = [2, 0.25, 0.25, 15] # time, lat, lon, depth

# define target datasets
targets = {
    # dissolved Fe
    'tblKOK1606_Gradients1_Diss_Trace_Metal_Profile': {
        'variables': ['Fe_dissolved'], 
        'tolerances': tolerances
    },
    'tblMGL1704_Gradients2_Diss_Trace_Metal_Profile': {
        'variables': ['Fe_dissolved'], 
        'tolerances': tolerances
    },
    'tblMGL1704_Gradients2_Diss_Trace_Metal_Surface': {
        'variables': ['Fe_dissolved'], 
        'tolerances': tolerances
    },
    'tblKM1906_Gradients3_Diss_Trace_Metal_Profile': {
        'variables': ['Fe_dissolved'], 
        'tolerances': tolerances
    }
}

# join target data on the source dataframe
merged_df = pycmap.Sample(
    source=source_df, 
    targets=targets, 
    replaceWithMonthlyClimatolog=False
)

# merge all of the iron data into one column
fe_start = 'CMAP_Fe_dissolved_tblKOK1606_Gradients1_Diss_Trace_Metal_Profile'
fe_end = 'CMAP_Fe_dissolved_tblKM1906_Gradients3_Diss_Trace_Metal_Profile'
merged_df['Fe_dissolved'] = merged_df.loc[:, fe_start:fe_end].max(axis=1)

# trim the dataframe
merged_df = merged_df[source_cols + ['Fe_dissolved']]

# calculate ratio
merged_df['N:Fe'] = (merged_df['NO3_NO2'] * 1e-6) / (merged_df['Fe_dissolved'] * 1e-9)

merged_df


# collect ecotype and gene weights

In [ ]:
# pull out ecotype weights

with pd.option_context('mode.chained_assignment', None):
    pro_fe_eco_df = pro_ds.sel(Component=fe_clusters['pro']).TaxonWeight.to_series().reset_index()
    pro_fe_eco_df['Genus'] = 'Prochlorococcus'
    pro_fe_eco_df['Cluster'] = f"pro{fe_clusters['pro']}"
    pro_fe_eco_df['ClusterGroup'] = 'Fe'
    pro_n_eco_df = pro_ds.sel(Component=n_clusters['pro']).TaxonWeight.to_series().reset_index()
    pro_n_eco_df['Genus'] = 'Prochlorococcus'
    pro_n_eco_df['Cluster'] = f"pro{n_clusters['pro']}"
    pro_n_eco_df['ClusterGroup'] = 'N'
    syn_fe_eco_df = syn_ds.sel(Component=fe_clusters['syn']).TaxonWeight.to_series().reset_index()
    syn_fe_eco_df['Genus'] = 'Synechococcus'
    syn_fe_eco_df['Cluster'] = f"syn{fe_clusters['syn']}"
    syn_fe_eco_df['ClusterGroup'] = 'Fe'
    syn_n_eco_df = syn_ds.sel(Component=n_clusters['syn']).TaxonWeight.to_series().reset_index()
    syn_n_eco_df['Genus'] = 'Synechococcus'
    syn_n_eco_df['Cluster'] = f"syn{n_clusters['syn']}"
    syn_n_eco_df['ClusterGroup'] = 'N'
ecotype_df = pd.concat([pro_fe_eco_df, pro_n_eco_df, syn_fe_eco_df, syn_n_eco_df])

# swap out clade labels with display labels
clade_order = ['HLI', 'HLII', 'LLI', '5.1B-I', '5.1B-CRD1', '5.1A-II', '5.1A-III', '5.1A-IV', '5.1A-CRD2', '5.1A-UC-A-EnvC']
clade_labels = ['HLI', 'HLII', 'LLI', 'I', 'CRD1', 'II', 'III', 'IV', 'CRD2', 'UC-A']
ecotype_df['Clade'] = ecotype_df['Clade'].map(dict(zip(clade_order, clade_labels)))

ecotype_df


In [ ]:
# pull out gene weights

geneweight_df = pd.DataFrame()
# pull gene weights of all clusters
with pd.option_context('mode.chained_assignment', None):
    for genus in ['pro', 'syn']:
        ds = {'pro': pro_ds, 'syn': syn_ds}[genus]
        for clustergroup in ['Fe', 'N']:
            cluster = {'Fe': fe_clusters, 'N': n_clusters}[clustergroup][genus]
            cluster_geneweight_df = ds.sel(Component=cluster).GeneWeight.to_series().reset_index()
            medians = cluster_geneweight_df.groupby('Ortholog').GeneWeight.median()
            cluster_geneweight_df = cluster_geneweight_df[
                cluster_geneweight_df.Ortholog.isin(medians[~medians.eq(0.)].index)].reset_index(drop=True)
            cluster_geneweight_df['Nonzero'] = cluster_geneweight_df.GeneWeight != 0
            cluster_geneweight_df['% Bootstraps'] = cluster_geneweight_df['Ortholog'].map(
                cluster_geneweight_df.groupby(['Ortholog']).Nonzero.mean()) * 100
            cluster_geneweight_df['Genus'] = {'pro': 'Prochlorococcus', 'syn': 'Synechococcus'}[genus]
            cluster_geneweight_df['Cluster'] = f"{genus}{cluster}"
            cluster_geneweight_df['ClusterGroup'] = clustergroup
            cluster_geneweight_df['Annotation'] = cluster_geneweight_df['Ortholog'].map(ds['Annotation'].to_series())
            if len(geneweight_df) == 0:
                geneweight_df = cluster_geneweight_df
            else:
                geneweight_df = pd.concat([geneweight_df, cluster_geneweight_df])

median_gene_df = geneweight_df.groupby(
    ['Cluster', 'Genus', 'ClusterGroup', 'Ortholog', 'Annotation', '% Bootstraps']).GeneWeight.median().reset_index()
median_gene_df


In [ ]:
# set gene labels to be displayed in plot

median_gene_df['GeneLabel'] = median_gene_df.Ortholog.map(gene_names_df.set_index('Ortholog')['GeneLabel'])
median_gene_df['ExpressionResponse'] = median_gene_df.Ortholog.map(gene_names_df.set_index('Ortholog')['ExpressionResponse'])
# fill in unannotated genes with CyCOG label
median_gene_df.loc[median_gene_df.GeneLabel.isna(), 'GeneLabel'] = median_gene_df.loc[median_gene_df.GeneLabel.isna(), 'Ortholog']
# pull out top genes
top_genes = median_gene_df[np.abs(median_gene_df['GeneWeight']).ge(0.04)].Ortholog.unique()
# append the tauA taurine transporter of another CyCOG
top_genes = np.append(top_genes, [60007639])

# look at how included genes
print(len(top_genes))
median_gene_df[median_gene_df['Ortholog'].isin(top_genes)].sort_values(
    ['ClusterGroup', 'Cluster', 'GeneWeight'], ascending=[True, True, False])


# panels a & b

- A: Ecotype Weights
- B: Gene Weights

In [ ]:
# make ecotype & gene weight figure

# set up figure
sample_weight_range = (-0.05, 1.05) # range of weights to show on sample plots
fig = plt.figure(figsize=(20, 4))
rows = 4
cols = 20
grid = plt.GridSpec(rows, cols, wspace=.2, hspace=.2)

# labels and colors
clusters = [f"pro{fe_clusters['pro']}", f"syn{fe_clusters['syn']}", f"pro{n_clusters['pro']}", f"syn{n_clusters['syn']}"]
genus_palette = dict(zip(['Prochlorococcus', 'Synechococcus'], [sns.color_palette()[i] for i in [2, 3]]))
clade_order = ['HLI', 'HLII', 'LLI', 'I', 'CRD1', 'II', 'III', 'IV', 'CRD2', 'UC-A']

# plot taxon weights
for i, cid in enumerate(clusters):
    axis = plt.subplot(grid[i, 1:6])
    plot_df = ecotype_df[ecotype_df['Cluster'] == cid]
    sns.boxplot(data=plot_df, x='Clade', y='TaxonWeight', hue='Genus', order=clade_order, hue_order=['Prochlorococcus', 'Synechococcus'], 
                palette=genus_palette, fill=False, whis=(5, 95), fliersize=1, 
                # legend=(i == n_clusters-1), 
                legend=False, 
                ax=axis)
    # labels and such
    axis.annotate(cid, xy=(-0.3, 0.5), xycoords='axes fraction', va='center', ha='left', fontsize=24, 
                  annotation_clip=False, color=genus_palette[plot_df.Genus.unique()[0]])
    axis.set(title=('Ecotype Weights' if not i else None), ylim=sample_weight_range, xlabel=None, ylabel=None)
    if i != 3:
        axis.tick_params(bottom=False, labelbottom=False)
    else:
        axis.set_xticks(clade_order, clade_order, rotation=90)

# plot gene weights
axis = plt.subplot(grid[:4, 6:])
axis.spines[['left', 'bottom']].set_visible(False)
plot_df = median_gene_df[median_gene_df['Ortholog'].isin(top_genes)].sort_values(
    ['ClusterGroup', 'Cluster', 'GeneWeight'], ascending=[True, True, False]).reset_index(drop=True)
n_genes = plot_df.GeneLabel.nunique()
with pd.option_context('mode.chained_assignment', None):
    plot_df['x'] = plot_df.GeneLabel.map(dict(zip(plot_df.GeneLabel.unique(), np.arange(n_genes))))
    plot_df['y'] = plot_df.Cluster.map(dict(zip(clusters, reversed(np.arange(len(clusters))))))
sns.scatterplot(x='x', y='y', hue='GeneWeight', 
    # size='% Bootstraps', 
    # sizes=(50, 500), 
    s=400, edgecolor=None, palette=bi_cmap, data=plot_df, legend=True, ax=axis
);
# plot marker gene status
axis.plot('x', 'y', '.', data=plot_df, markerfacecolor='w', markeredgecolor='none', zorder=0, label=' ')
axis.plot('x', 'y', 'ks', data=plot_df[plot_df['ExpressionResponse'] == 'Fe responsive'], markeredgewidth=3, 
          markerfacecolor='none', ms=26, markeredgecolor=sns.color_palette()[1], label='Fe responsive')
axis.plot('x', 'y', '.', data=plot_df, markerfacecolor='w', markeredgecolor='none', zorder=0, label='  ')
axis.plot('x', 'y', 'ks', data=plot_df[plot_df['ExpressionResponse'] == 'N responsive'], markeredgewidth=3, 
          markerfacecolor='none', ms=26, markeredgecolor=sns.color_palette()[7], label='N responsive')

axis.set(title='Gene Weights', xlabel=None, ylabel=None, xlim=(-0.5, n_genes-0.5), ylim=(-0.5, 3.5))
axis.set_xticks(np.arange(n_genes), plot_df.GeneLabel.unique(), rotation=90)
axis.tick_params(left=False, labelleft=False, bottom=False)
axis.legend(title='     Median\nGene Weight', loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()


# panels c & d

- C: Surface Samples
- D: Incubations

In [ ]:
# set up incubation weights with both Fe and nitrogen clusters

# set up data
treatments = ['control 0h', 'control 96h', 'low Fe', 'high Fe', 'Fe', 'N+P+Fe', 'N+P', 'low N+P', 'high N+P']
with pd.option_context('mode.chained_assignment', None):
    fe_incubation_df = fe_cluster_df[fe_cluster_df.Treatment.isin(treatments)]
    fe_incubation_df['ClusterGroup'] = 'Fe'
    n_incubation_df = n_cluster_df[n_cluster_df.Treatment.isin(treatments)]
    n_incubation_df['ClusterGroup'] = 'N'
incubation_df = pd.concat([fe_incubation_df, n_incubation_df]).reset_index(drop=True).fillna(0)
incubation_df = incubation_df.sort_values('Treatment', key=lambda s: s.apply(treatments.index))
incubation_df['Condition'] = incubation_df['Condition'].map({'gyre': 'Gyre', 'TZ': 'TZ', 'HNLC': 'HNLC'})

incubation_df


In [ ]:
# build combined fe & n cluster figure

# set up figure
sample_weight_range = (-0.05, 0.7) # range of weights to show on sample plots
fig = plt.figure(figsize=(24, 12))
rows = 7
cols = 27
grid = plt.GridSpec(rows, cols, hspace=.3, wspace=.3)
n_bins = 21
bounds = (23, 43)
depth_limit = 15 # meters
# time_classes = ['2am', '6am', '10am']
time_classes = ['2am', '6am', '10am', '2pm', '6pm', '10pm']
fliersize = 1
annot_kwgs = {
    'xycoords': 'axes fraction', 'va': 'center', 'ha': 'left', 'fontsize': 26, 'annotation_clip': False
}

# panel c: surface transects + environmental covariates
for i, year in enumerate([2016, 2017, 2019]):
    # Fe limitation cluster
    axis = plt.subplot(grid[:, 7*i:7*i+2])
    # add annotation
    if i == 0:
        axis.annotate('C', xy=(-0.5, 1.1), **annot_kwgs)
    # get data together
    plot_df = fe_cluster_df[fe_cluster_df['SampleGroup'] == 'surface transect']
    plot_df = plot_df[plot_df['TimeClass'].isin(time_classes)]
    plot_df = plot_df[plot_df['Year'] == year]
    latitudes = plot_df[
        ['SampleName', 'Latitude']].drop_duplicates().set_index('SampleName').sort_values('Latitude')['Latitude']
    lat_bins = unique_bins(latitudes, n_bins, bounds=bounds, reverse=False)
    plot_df['lat_idx'] = plot_df['SampleName'].map(dict(zip(latitudes.index, lat_bins)))
    # make figure
    sns.boxplot(
        data=plot_df, x='SampleWeight', y='lat_idx', hue='Genus', hue_order=['Prochlorococcus', 'Synechococcus'], 
        palette=[sns.color_palette()[i] for i in [2, 3]], orient='y', order=np.arange(n_bins), 
        fill=False, whis=(5, 95), fliersize=fliersize, legend=False, ax=axis
    )
    # set labels and such
    axis.set_yticks(np.arange(1, n_bins, 2), [f'{int(l)}N' for l in np.arange(bounds[0]+1, bounds[1]+1, 2)])
    axis.set(xlim=sample_weight_range, ylim=(-0.5, n_bins-0.5), xlabel='Weight', ylabel='', title='Fe Clusters')
    axis.set_yticks(np.arange(n_bins), minor=True)
    
    # N acquisition cluster
    axis = plt.subplot(grid[:, 7*i+2:7*i+4])
    # add title
    if i == 1:
        axis.annotate('Surface Transects', xy=(.5, 1.1), xycoords='axes fraction', va='center', ha='center', fontsize=22)
    # get data together
    plot_df = n_cluster_df[n_cluster_df['SampleGroup'] == 'surface transect']
    plot_df = plot_df[plot_df['TimeClass'].isin(time_classes)]
    plot_df = plot_df[plot_df['Year'] == year]
    plot_df['lat_idx'] = plot_df['SampleName'].map(dict(zip(latitudes.index, lat_bins)))
    # make figure
    sns.boxplot(
        data=plot_df, x='SampleWeight', y='lat_idx', hue='Genus', hue_order=['Prochlorococcus', 'Synechococcus'], 
        palette=[sns.color_palette()[i] for i in [2, 3]], orient='y', order=np.arange(n_bins), fill=False, 
        whis=(5, 95), fliersize=fliersize, legend=False, ax=axis
    )
    # set labels and such
    axis.set_yticks(np.arange(1, n_bins, 2))
    axis.set(xlim=sample_weight_range, ylim=(-0.5, n_bins-0.5), xlabel='Weight', ylabel='', title=f"{year}\nN Clusters")
    axis.set_yticks(np.arange(n_bins), minor=True)
    axis.tick_params(left=True, labelleft=False)

    # environmental covariates
    axis = plt.subplot(grid[:, 7*i+4:7*i+6])
    # plot salinity front
    axis.hlines(y=sal_fronts[year], xmin=0, xmax=2, linewidth=10, color=sns.color_palette()[5], alpha=0.2, label='Salinity Front')
    # plot chlorophyll front
    axis.hlines(y=chl_fronts[year], xmin=0, xmax=2, linewidth=10, color=sns.color_palette()[2], alpha=0.2, label='Chlorophyll Front')
    
    # dissolved iron
    color = sns.color_palette()[1]
    sns.lineplot(
        data=fe_df[(fe_df['year'] == year) & (fe_df['depth'].le(depth_limit))], x='Fe_dissolved', 
        y='lat', orient='y', ax=axis, color=color, marker='.', label='[Fe] (nmol/L)', legend=(i==2)
    )
    axis.set(xlim=(-.05, 0.85), ylim=(bounds[0]-0.5, bounds[1]+.5), yticks=np.arange(1, n_bins, 2)+bounds[0], 
             xlabel='[Fe]', ylabel='')
    axis.set_yticks(np.arange(n_bins)+bounds[0], minor=True)
    axis.tick_params(left=True, labelleft=False)
    axis.tick_params(axis='x', colors=color)
    axis.xaxis.label.set_color(color)
    axis.spines['bottom'].set_color(color)
    if i == 2:
        axis.legend(loc='upper left', bbox_to_anchor=(2.3, -0.06))

    # dissolved nitrate + nitrite
    color = sns.color_palette()[7]
    axis2 = plt.twiny(axis)
    sns.lineplot(
        data=n_df[(n_df['year'] == year) & (n_df['depth'].le(depth_limit))], x='NO3_NO2', y='lat', orient='y', 
        ax=axis2, color=color, marker='.', label='[NO$_3^-$+NO$_2^-$] (umol/L)', legend=(i==2)
    )
    axis2.set(xlim=(-0.5, 7.5), ylim=(bounds[0]-0.5, bounds[1]+.5), yticks=np.arange(1, n_bins, 2)+bounds[0], 
             xlabel='[NO$_3^-$+NO$_2^-$]', ylabel='')
    axis2.set_yticks(np.arange(n_bins)+bounds[0], minor=True)
    axis2.tick_params(top=False, labeltop=False, bottom=True, labelbottom=True)
    axis2.tick_params(axis='x', colors=color)
    axis2.xaxis.set_label_position('bottom')
    axis2.xaxis.label.set_color(color)
    axis2.spines['bottom'].set_color(color)
    axis2.spines['bottom'].set_position(('outward', 60))
    if i == 2:
        axis2.legend(loc='upper left', bbox_to_anchor=(2.3, -0.165))

    # N:Fe ratio
    color = sns.color_palette()[4]
    axis3 = plt.twiny(axis2)
    sns.lineplot(
        data=merged_df[(merged_df['year'] == year) & (merged_df['depth'].le(depth_limit))], x='N:Fe', y='lat', orient='y', 
        ax=axis3, color=color, marker='.', label='[NO$_3^-$+NO$_2^-$]:[Fe] Ratio', legend=(i==2)
    )
    axis3.set(xlim=(1, 1000000), ylim=(bounds[0]-0.5, bounds[1]+.5), yticks=np.arange(1, n_bins, 2)+bounds[0], 
             xlabel='[NO$_3^-$+NO$_2^-$]:[Fe]', ylabel='', xscale='log')
    axis3.set_yticks(np.arange(n_bins)+bounds[0], minor=True)
    axis3.tick_params(top=False, labeltop=False, bottom=True, labelbottom=True)
    axis3.tick_params(axis='x', colors=color)
    axis3.xaxis.set_label_position('bottom')
    axis3.xaxis.label.set_color(color)
    axis3.spines['bottom'].set_color(color)
    axis3.spines['bottom'].set_position(('outward', 120))
    if i == 2:
        axis3.legend(loc='upper left', bbox_to_anchor=(2.3, -0.2))

# panel d: incubation weights
for i, condition in enumerate(['HNLC', 'TZ', 'Gyre']):
    for j, clustergroup in enumerate(['Fe', 'N']):
        axis = plt.subplot(grid[i*2:i*2+2, 23+(2*j):25+(2*j)])
        plot_df = incubation_df[(incubation_df['Condition'] == condition) & (incubation_df['ClusterGroup'] == clustergroup)]
        sns.boxplot(
            data=plot_df, x='SampleWeight', y='Treatment', hue='Genus', 
            hue_order=['Prochlorococcus', 'Synechococcus'], palette=[sns.color_palette()[i] for i in [2, 3]], 
            orient='y', fill=False, whis=(5, 95), fliersize=fliersize, legend=(i==2 and j==0), ax=axis
        )
        # labels and such
        if i == 0:
            axis.set(title='{} Clusters'.format(clustergroup))
            if j == 0:
                axis.annotate('D', xy=(-1.5, 1.35), **annot_kwgs)
            if j == 1:
                axis.annotate('Incubations (2017)', xy=(0, 1.35), xycoords='axes fraction', va='center', ha='center', fontsize=22)
        if i == 2:
            xlabel = 'Weight'
            if j == 0:
                axis.legend(loc='upper left', bbox_to_anchor=(-0.52, -0.4))
        else:
            xlabel = None
        if j == 0:
            ylabel = f"{condition} ({plot_df.Latitude.mean():.0f}N)"
        else:
            ylabel = None
        axis.set(xlim=sample_weight_range, xlabel=xlabel, ylabel=ylabel)
        axis.tick_params(left=True, labelleft=(j==0), bottom=True, labelbottom=(i==2))

plt.show()


# full combined figure

In [ ]:
# build combined fe & n cluster figure

# set up figure
fig = plt.figure(figsize=(24, 20))
rows = 17
cols = 27
grid = plt.GridSpec(rows, cols, hspace=.3, wspace=.3)
n_bins = 21
bounds = (23, 43)
depth_limit = 15 # meters
# time_classes = ['2am', '6am', '10am']
time_classes = ['2am', '6am', '10am', '2pm', '6pm', '10pm']
fliersize = 1
annot_kwgs = {
    'xycoords': 'axes fraction', 'va': 'center', 'ha': 'left', 'fontsize': 26, 'annotation_clip': False
}

# labels and colors
clusters = [f"pro{fe_clusters['pro']}", f"syn{fe_clusters['syn']}", f"pro{n_clusters['pro']}", f"syn{n_clusters['syn']}"]
genus_palette = dict(zip(['Prochlorococcus', 'Synechococcus'], [sns.color_palette()[i] for i in [2, 3]]))
clade_order = ['HLI', 'HLII', 'LLI', 'I', 'CRD1', 'II', 'III', 'IV', 'CRD2', 'UC-A']

# panel a: taxon weights
sample_weight_range = (-0.05, 1.05) # range of weights to show on sample plots
for i, cid in enumerate(clusters):
    axis = plt.subplot(grid[i, 2:8])
    # add annotation
    if i == 0:
        axis.annotate('A', xy=(-0.5, 1.6), **annot_kwgs)
    elif i == 1: 
        axis.annotate('Fe Clusters', xy=(-0.4, 1), xycoords='axes fraction', va='center', ha='center', rotation=90, fontsize=18)
    elif i == 3:
        axis.annotate('N Clusters', xy=(-0.4, 1), xycoords='axes fraction', va='center', ha='center', rotation=90, fontsize=18)
    plot_df = ecotype_df[ecotype_df['Cluster'] == cid]
    sns.boxplot(data=plot_df, x='Clade', y='TaxonWeight', hue='Genus', order=clade_order, hue_order=['Prochlorococcus', 'Synechococcus'], 
                palette=genus_palette, fill=False, whis=(5, 95), fliersize=1, 
                # legend=(i == n_clusters-1), 
                legend=False, 
                ax=axis)
    # labels and such
    axis.annotate(cid, xy=(-0.3, 0.5), xycoords='axes fraction', va='center', ha='left', fontsize=22, 
                  annotation_clip=False, color=genus_palette[plot_df.Genus.unique()[0]])
    axis.set(title=('Ecotype Weights' if not i else None), ylim=sample_weight_range, xlabel=None, ylabel=None)
    if i != 3:
        axis.tick_params(bottom=False, labelbottom=False)
    else:
        axis.set_xticks(clade_order, clade_order, rotation=90)

# panel b: gene weights
axis = plt.subplot(grid[:4, 9:24])
axis.annotate('B', xy=(-0.05, 1.15), **annot_kwgs)
axis.spines[['left', 'bottom']].set_visible(False)
plot_df = median_gene_df[median_gene_df['Ortholog'].isin(top_genes)].sort_values(
    ['ClusterGroup', 'Cluster', 'GeneWeight'], ascending=[True, True, False]).reset_index(drop=True)
n_genes = plot_df.GeneLabel.nunique()
with pd.option_context('mode.chained_assignment', None):
    plot_df['x'] = plot_df.GeneLabel.map(dict(zip(plot_df.GeneLabel.unique(), np.arange(n_genes))))
    plot_df['y'] = plot_df.Cluster.map(dict(zip(clusters, reversed(np.arange(len(clusters))))))
sns.scatterplot(x='x', y='y', hue='GeneWeight', 
    # size='% Bootstraps', 
    # sizes=(50, 500), 
    s=400, edgecolor=None, palette=bi_cmap, data=plot_df, legend=True, ax=axis
);
# plot marker gene status
axis.plot('x', 'y', '.', data=plot_df, markerfacecolor='w', markeredgecolor='none', zorder=0, label=' ')
axis.plot('x', 'y', 'ks', data=plot_df[plot_df['ExpressionResponse'] == 'Fe responsive'], markeredgewidth=3, 
          markerfacecolor='none', ms=26, markeredgecolor=sns.color_palette()[1], label='Fe responsive')
axis.plot('x', 'y', '.', data=plot_df, markerfacecolor='w', markeredgecolor='none', zorder=0, label='  ')
axis.plot('x', 'y', 'ks', data=plot_df[plot_df['ExpressionResponse'] == 'N responsive'], markeredgewidth=3, 
          markerfacecolor='none', ms=26, markeredgecolor=sns.color_palette()[7], label='N responsive')

axis.set(title='Gene Weights', xlabel=None, ylabel=None, xlim=(-0.6, n_genes-0.4), ylim=(-0.5, 3.5))
axis.set_xticks(np.arange(n_genes), plot_df.GeneLabel.unique(), rotation=90)
axis.tick_params(left=False, labelleft=False, bottom=False)
axis.legend(title='     Median\nGene Weight', loc='center left', bbox_to_anchor=(1, 0.5))


# panel c: surface transects + environmental covariates
sample_weight_range = (-0.05, 0.7) # range of weights to show on sample plots
for i, year in enumerate([2016, 2017, 2019]):
    # Fe limitation cluster
    axis = plt.subplot(grid[7:, 7*i:7*i+2])
    # add annotation
    if i == 0:
        axis.annotate('C', xy=(-0.5, 1.1), **annot_kwgs)
    # get data together
    plot_df = fe_cluster_df[fe_cluster_df['SampleGroup'] == 'surface transect']
    plot_df = plot_df[plot_df['TimeClass'].isin(time_classes)]
    plot_df = plot_df[plot_df['Year'] == year]
    latitudes = plot_df[
        ['SampleName', 'Latitude']].drop_duplicates().set_index('SampleName').sort_values('Latitude')['Latitude']
    lat_bins = unique_bins(latitudes, n_bins, bounds=bounds, reverse=False)
    plot_df['lat_idx'] = plot_df['SampleName'].map(dict(zip(latitudes.index, lat_bins)))
    # make figure
    sns.boxplot(
        data=plot_df, x='SampleWeight', y='lat_idx', hue='Genus', hue_order=['Prochlorococcus', 'Synechococcus'], 
        palette=[sns.color_palette()[i] for i in [2, 3]], orient='y', order=np.arange(n_bins), 
        fill=False, whis=(5, 95), fliersize=fliersize, legend=False, ax=axis
    )
    # set labels and such
    axis.set_yticks(np.arange(1, n_bins, 2), [f'{int(l)}N' for l in np.arange(bounds[0]+1, bounds[1]+1, 2)])
    axis.set(xlim=sample_weight_range, ylim=(-0.5, n_bins-0.5), xlabel='Weight', ylabel='', title='Fe Clusters')
    axis.set_yticks(np.arange(n_bins), minor=True)
    
    # N acquisition cluster
    axis = plt.subplot(grid[7:, 7*i+2:7*i+4])
    # add title
    if i == 1:
        axis.annotate('Surface Transects', xy=(.5, 1.1), xycoords='axes fraction', va='center', ha='center', fontsize=22)
    # get data together
    plot_df = n_cluster_df[n_cluster_df['SampleGroup'] == 'surface transect']
    plot_df = plot_df[plot_df['TimeClass'].isin(time_classes)]
    plot_df = plot_df[plot_df['Year'] == year]
    plot_df['lat_idx'] = plot_df['SampleName'].map(dict(zip(latitudes.index, lat_bins)))
    # make figure
    sns.boxplot(
        data=plot_df, x='SampleWeight', y='lat_idx', hue='Genus', hue_order=['Prochlorococcus', 'Synechococcus'], 
        palette=[sns.color_palette()[i] for i in [2, 3]], orient='y', order=np.arange(n_bins), fill=False, 
        whis=(5, 95), fliersize=fliersize, legend=False, ax=axis
    )
    # set labels and such
    axis.set_yticks(np.arange(1, n_bins, 2))
    axis.set(xlim=sample_weight_range, ylim=(-0.5, n_bins-0.5), xlabel='Weight', ylabel='', title=f"{year}\nN Clusters")
    axis.set_yticks(np.arange(n_bins), minor=True)
    axis.tick_params(left=True, labelleft=False)

    # environmental covariates
    axis = plt.subplot(grid[7:, 7*i+4:7*i+6])
    # plot salinity front
    axis.hlines(y=sal_fronts[year], xmin=0, xmax=2, linewidth=10, color=sns.color_palette()[5], alpha=0.2, label='Salinity Front')
    # plot chlorophyll front
    axis.hlines(y=chl_fronts[year], xmin=0, xmax=2, linewidth=10, color=sns.color_palette()[2], alpha=0.2, label='Chlorophyll Front')
    
    # dissolved iron
    color = sns.color_palette()[1]
    sns.lineplot(
        data=fe_df[(fe_df['year'] == year) & (fe_df['depth'].le(depth_limit))], x='Fe_dissolved', 
        y='lat', orient='y', ax=axis, color=color, marker='.', label='[Fe] (nmol/L)', legend=(i==2)
    )
    axis.set(xlim=(-.05, 0.85), ylim=(bounds[0]-0.5, bounds[1]+.5), yticks=np.arange(1, n_bins, 2)+bounds[0], 
             xlabel='[Fe]', ylabel='')
    axis.set_yticks(np.arange(n_bins)+bounds[0], minor=True)
    axis.tick_params(left=True, labelleft=False)
    axis.tick_params(axis='x', colors=color)
    axis.xaxis.label.set_color(color)
    axis.spines['bottom'].set_color(color)
    if i == 2:
        axis.legend(loc='upper left', bbox_to_anchor=(2.3, -0.1))

    # dissolved nitrate + nitrite
    color = sns.color_palette()[7]
    axis2 = plt.twiny(axis)
    sns.lineplot(
        data=n_df[(n_df['year'] == year) & (n_df['depth'].le(depth_limit))], x='NO3_NO2', y='lat', orient='y', 
        ax=axis2, color=color, marker='.', label='[NO$_3^-$+NO$_2^-$] (umol/L)', legend=(i==2)
    )
    axis2.set(xlim=(-0.5, 7.5), ylim=(bounds[0]-0.5, bounds[1]+.5), yticks=np.arange(1, n_bins, 2)+bounds[0], 
             xlabel='[NO$_3^-$+NO$_2^-$]', ylabel='')
    axis2.set_yticks(np.arange(n_bins)+bounds[0], minor=True)
    axis2.tick_params(top=False, labeltop=False, bottom=True, labelbottom=True)
    axis2.tick_params(axis='x', colors=color)
    axis2.xaxis.set_label_position('bottom')
    axis2.xaxis.label.set_color(color)
    axis2.spines['bottom'].set_color(color)
    axis2.spines['bottom'].set_position(('outward', 60))
    if i == 2:
        axis2.legend(loc='upper left', bbox_to_anchor=(2.3, -0.205))

    # N:Fe ratio
    color = sns.color_palette()[4]
    axis3 = plt.twiny(axis2)
    sns.lineplot(
        data=merged_df[(merged_df['year'] == year) & (merged_df['depth'].le(depth_limit))], x='N:Fe', y='lat', orient='y', 
        ax=axis3, color=color, marker='.', label='[NO$_3^-$+NO$_2^-$]:[Fe] Ratio', legend=(i==2)
    )
    axis3.set(xlim=(1, 1000000), ylim=(bounds[0]-0.5, bounds[1]+.5), yticks=np.arange(1, n_bins, 2)+bounds[0], 
             xlabel='[NO$_3^-$+NO$_2^-$]:[Fe]', ylabel='', xscale='log')
    axis3.set_yticks(np.arange(n_bins)+bounds[0], minor=True)
    axis3.tick_params(top=False, labeltop=False, bottom=True, labelbottom=True)
    axis3.tick_params(axis='x', colors=color)
    axis3.xaxis.set_label_position('bottom')
    axis3.xaxis.label.set_color(color)
    axis3.spines['bottom'].set_color(color)
    axis3.spines['bottom'].set_position(('outward', 120))
    if i == 2:
        axis3.legend(loc='upper left', bbox_to_anchor=(2.3, -0.24))

# panel d: incubation weights
for i, condition in enumerate(['HNLC', 'TZ', 'Gyre']):
    for j, clustergroup in enumerate(['Fe', 'N']):
        axis = plt.subplot(grid[i*3+7:i*3+10, 23+(2*j):25+(2*j)])
        plot_df = incubation_df[(incubation_df['Condition'] == condition) & (incubation_df['ClusterGroup'] == clustergroup)]
        sns.boxplot(
            data=plot_df, x='SampleWeight', y='Treatment', hue='Genus', 
            hue_order=['Prochlorococcus', 'Synechococcus'], palette=[sns.color_palette()[i] for i in [2, 3]], 
            orient='y', fill=False, whis=(5, 95), fliersize=fliersize, legend=(i==2 and j==0), ax=axis
        )
        # labels and such
        if i == 0:
            axis.set(title='{} Clusters'.format(clustergroup))
            if j == 0:
                axis.annotate('D', xy=(-1.5, 1.35), **annot_kwgs)
            if j == 1:
                axis.annotate('Incubations (2017)', xy=(0, 1.35), xycoords='axes fraction', va='center', ha='center', fontsize=22)
        if i == 2:
            xlabel = 'Weight'
            if j == 0:
                axis.legend(loc='upper left', bbox_to_anchor=(-0.52, -0.35))
        else:
            xlabel = None
        if j == 0:
            ylabel = f"{condition} ({plot_df.Latitude.mean():.0f}N)"
        else:
            ylabel = None
        axis.set(xlim=sample_weight_range, xlabel=xlabel, ylabel=ylabel)
        axis.tick_params(left=True, labelleft=(j==0), bottom=True, labelbottom=(i==2))

fig.savefig(outdir / 'fe-n-clusters.png', bbox_inches='tight')
